# Лабораторная №5
## Васильева Екатерина
## Группа M32081

In [164]:
import numpy as np
import pandas as pd
import scipy.stats as st
from scipy import stats

from collections import Counter

In [165]:
df = pd.read_csv('MEN_SHOES.csv')
df

,Brand_Name,How_Many_Sold,Current_Price,Product_details,RATING
0,ASIAN,"2,242","₹1,098","Oxygen-01 Sports Running,Walking & Gym Shoes w...",3.8
1,ASIAN,240,₹674,"Men's Express-08 Sports Running,Walking,Gym,Tr...",4.0
2,ASIAN,"16,662",₹588,"Men's Cosko Sports Running,Walking,Gym,Trainin...",3.8
3,ASIAN,135,₹599,"Wind-03 Sports Running,Walking & Gym Shoes wit...",4.0
4,Reebok,240,₹982,Men's Velocity Runner Lp Running Shoe,4.0
...,...,...,...,...,...
23935,Campus,198,₹623,Men's HARVEL PRO Running Shoes,4.0
23936,Wakefield,49,₹399,Stylish Running Sports Shoes for MEN-2010,3.4
23937,URJO,557,₹839,Mens Running Shoes,4.2
23938,FEETEES,63,₹349,Nexon Men's Casual Eva Socks Knitted Running S...,3.6


In [166]:
df = df.dropna()

In [167]:
df['How_Many_Sold']=df['How_Many_Sold'].apply(lambda x: int(x.replace(',','')))

C:\Users\ASUS\AppData\Local\Temp\ipykernel_10036\3847641309.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['How_Many_Sold']=df['How_Many_Sold'].apply(lambda x: int(x.replace(',','')))


In [168]:
df['Current_Price']=df['Current_Price'].apply(lambda x: int(x.replace('₹','').replace(',','')))

C:\Users\ASUS\AppData\Local\Temp\ipykernel_10036\1144419968.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Current_Price']=df['Current_Price'].apply(lambda x: int(x.replace('₹','').replace(',','')))


In [169]:
class LinearRegression():
    def __init__(self, df, features, target, bias = False, coef_mask = None):
        self.df = df
        self.features = features
        self.target = target
        self.masking = Counter(coef_mask)[0] # Если модель с ограничениями, здесь будет храниться их количество
        self.mask = coef_mask

        if bias:
            self.features.insert(0, 'bias')
            if 'bias' not in df.columns:
                df.insert(0, 'bias', [1] * len(self.df))
            if self.masking:
                coef_mask = np.insert(coef_mask, 0, 1)

        self.X = self.df[features]

        if self.masking:
            columns = list(self.X.columns)
            remained_columns = []
            for i, mask in enumerate(coef_mask):
                if mask:
                    remained_columns.append(columns[i])
            X = self.X[remained_columns].to_numpy()
            self.y = self.df[target].to_numpy()
            self.coefs = self.make_model(X, self.y)
            for idx, mask in enumerate(coef_mask):
                if not mask:
                    self.coefs = np.insert(self.coefs, idx, 0)

        else:
            self.X = self.X.to_numpy()
            self.y = self.df[target].to_numpy()
            self.coefs = self.make_model(self.X, self.y)


    def make_model(self, X, y):
        coefs = np.linalg.inv(X.T @ X) @ X.T @ y
        return coefs


    def predict(self, x):
        return x @ self.coefs


    def residual_variance(self):
        predictions = self.X @ self.coefs
        eps = predictions - self.y
        error = eps @ eps.T
        n, m = self.X.shape
        return error / (n - m)


    def count_inv_A(self):
        A = self.X.T @ self.X
        invA = np.linalg.inv(A)
        return invA


    def _count_term(self, alpha):
        invA = self.count_inv_A()
        # Подсчет доверительного интервала
        n, m = self.X.shape
        quantile = st.t.ppf(1 - alpha / 2, df = n - m)

        S2 = self.residual_variance()
        term = S2 ** 0.5
        term *= quantile
        return term, invA


    def count(self):
        return self.X @ self.coefs


    def name_interval(self, name, alpha = 0.05):
        term, invA = self._count_term(alpha)
        idx = self.features.index(name)
        coef = self.coefs[idx]
        current_term = term * np.sqrt(invA[idx][idx])
        left_bound = coef - current_term
        right_bound = coef + current_term

        return left_bound, right_bound


    def all_intervals(self, alpha = 0.05):
        term, invA = self._count_term(alpha)
        left_bounds = []
        right_bounds = []

        for i, coef in enumerate(self.coefs):
            current_term = term * np.sqrt(invA[i][i])
            left_bound = coef - current_term
            right_bound = coef + current_term
            left_bounds.append(left_bound)
            right_bounds.append(right_bound)

        return left_bounds, right_bounds


    def print_info(self, alpha = 0.05):
        info = '\n'.join([f'Коэффициент для {self.features[i]}: {self.coefs[i]}'
                          for i in range(len(self.coefs))])
        print(info)

        print('\nОстаточная дисперсия : ', self.residual_variance())

        print('\nКоэффициент детерминации: ', self.determination_coef())

        print(f'\nДоверительные интервалы для alpha = {alpha}:')
        left_bounds, right_bounds = self.all_intervals(alpha)
        info = '\n'.join([
            '{}: [{}; {}]'.
            format(self.features[i], left_bounds[i], right_bounds[i])
            for i in range(len(self.features))])
        print(info)


    def determination_coef(self):
        sse = sum((self.y - self.X @ self.coefs) ** 2)
        mean_y = np.mean(self.y)
        sst = sum((self.y - mean_y) ** 2)
        return 1 - sse / sst


    def countT_stat(self, name):

        n, m = self.X.shape
        idx = self.features.index(name)
        coef = self.coefs[idx]
        num = coef
        invA = self.count_inv_A()
        S2 = self.residual_variance()
        denum = (invA[idx][idx] * S2) ** 0.5
        T = num / denum
        p_value = 1 - st.t.cdf(T, n - m)
        return T, p_value

LinearRegression() по сути является оберткой над различными реализациями методов наименьших квадратов.

***Задание 1***

Вектор независимых переменных - $x=(x_0,x_1,x_2)$ <br>
$x_0 = 1$, так как модель со свободным коэффициентом
$y_i \approx \sum_{j=1}^4 x_{ij}c_j$ <br>
$y$ - целевая переменная

In [170]:
lr = LinearRegression(df = df,
                      features = ['How_Many_Sold', 'Current_Price'],
                      target = 'RATING',
                      bias = True)
lr.print_info()

Коэффициент для bias: 3.347571501914359
Коэффициент для How_Many_Sold: 8.263894533593866e-06
Коэффициент для Current_Price: 0.0005263263156671574

Остаточная дисперсия :  0.11973197812338982

Коэффициент детерминации:  0.26872791897558723

Доверительные интервалы для alpha = 0.05:
bias: [3.336485457063946; 3.3586575467647717]
How_Many_Sold: [7.849699469747399e-06; 8.678089597440333e-06]
Current_Price: [0.000514679493923359; 0.0005379731374109557]


Коэффициент детерминациии 0.3 < 0.5, что низко и показывает на низкую зависимость независимых переменных. По коэффициентам видим положительную корреляцию рейтинга от количества продаж и цены. Свободный коэффициент - влияние на рейтинг остальных невыбранных факторов

***Задание 2***
а) Чем больше продажи, тем больше рейтинг

$H_0$ - рейтинг не зависит от продаж
$H_1$ - рейтинг зависит от продаж

In [171]:
stat, p = lr.countT_stat(name = 'How_Many_Sold')
if p > 0.05:
    print("H0")
else:
    print("H1")

H1


б)Рейтинг зависит от цены

$H_0$ - нет никакой зависимости
$H_1$ - есть зависимость

In [172]:
stat, p = lr.countT_stat(name = 'Current_Price')
if p > 0.05:
    print("H0")
else:
    print("H1")

H1


в) Рейтинг зависит и от цены, и от количества проданных экземпляров

$H_0$ - нет такой зависимости
$H_1$ - есть зависимость


In [173]:
lr1 = LinearRegression(df = df,
                          features = ['How_Many_Sold', 'Current_Price'],
                          target = 'RATING',
                          bias = True,
                          coef_mask = [0, 0])
lr1.print_info()

Коэффициент для bias: 3.820689655172416
Коэффициент для How_Many_Sold: 0.0
Коэффициент для Current_Price: 0.0

Остаточная дисперсия :  0.1637310943905723

Коэффициент детерминации:  -4.3298697960381105e-14

Доверительные интервалы для alpha = 0.05:
bias: [3.8077257115177243; 3.833653598827108]
How_Many_Sold: [-4.843568235750977e-07; 4.843568235750977e-07]
Current_Price: [-1.361971224906404e-05; 1.361971224906404e-05]


In [174]:
def countF_stat(lr1: LinearRegression,
                lr: LinearRegression):
    temp = lr1.count()
    cond_error = temp.T @ temp
    temp = lr.count() - lr.y
    error = temp.T @ temp
    F = (cond_error - error) / error
    n, m = lr.df.shape
    F *= (n - m) / lr1.masking
    p_value = 1 - st.f.cdf(F, lr1.masking, n - m)
    return F, p_value

In [175]:
stat, p = countF_stat(lr1, lr)
if p > 0.05:
    print("H0")
else:
    print("H1")

H1
